In [1]:
import pandas as pd

In [2]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
len(df_jan.columns)

19

In [5]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [6]:
df_jan['duration'] = df_jan.apply(lambda row: pd.Timedelta(row['tpep_dropoff_datetime'] - row['tpep_pickup_datetime']).total_seconds() / 60.0, axis=1)

In [7]:
df_jan['duration'].head()

0     8.433333
1     6.316667
2    12.750000
3     9.616667
4    10.833333
Name: duration, dtype: float64

In [8]:
df_jan['duration'].std()

42.594351241920904

In [9]:
len(df_jan.query('1 <= duration <= 60'))/len(df_jan)

0.9812202822125979

In [10]:
df_jan = df_jan.query('1 <= duration <= 60')

In [11]:
locationsIDs = df_jan[['PULocationID','DOLocationID']].astype(str).to_dict('records')

In [12]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()

In [13]:
dv.fit(locationsIDs)
transformed_data=dv.transform(locationsIDs)
transformed_data

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [14]:
transformed_data.shape

(3009173, 515)

In [15]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [16]:
y = df_jan['duration'].to_numpy()
y

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [17]:
model.fit(transformed_data, y)

LinearRegression()

In [18]:
from sklearn.metrics import mean_squared_error

In [19]:
y_pred = model.predict(transformed_data)

In [20]:
rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
rmse

7.649261027792376

In [21]:
locationsIDs=None
transformed_data=None

In [22]:
df_feb['duration'] = df_feb.apply(lambda row: pd.Timedelta(row['tpep_dropoff_datetime'] - row['tpep_pickup_datetime']).total_seconds() / 60.0, axis=1)

In [23]:
df_feb = df_feb.query('1 <= duration <= 60')

In [24]:
locationsIDs = df_feb[['PULocationID','DOLocationID']].astype(str).to_dict('records')

In [25]:
dv.fit(locationsIDs)
transformed_data=dv.transform(locationsIDs)
transformed_data

<2855951x514 sparse matrix of type '<class 'numpy.float64'>'
	with 5711902 stored elements in Compressed Sparse Row format>

In [26]:
transformed_data.shape

(2855951, 514)

In [27]:
y = df_feb['duration'].to_numpy()
y

array([ 1.68333333, 32.08333333, 13.3       , ..., 14.        ,
        7.        ,  9.8       ])

In [28]:
y_pred = model.predict(transformed_data)

ValueError: X has 514 features, but LinearRegression is expecting 515 features as input.

In [ ]:
rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
rmse